### Imports

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

from taegis_magic.pandas.context import (
    normalize_entities,
    relate_entities, 
    generate_context_queries, 
    display_facets
)

%load_ext taegis_magic

### Set Context for Queries

In [ ]:
GROUP_BY = '@user' # or @ip/@domain/@hash/@host
REGION = 'charlie' # or delta/echo/foxtrot
TENANT = '50530'

### Search Entrypoint

In [ ]:
%%taegis alerts search --region $REGION --tenant $TENANT --assign alerts_dataframe
FROM alert
WHERE
    metadata.title = 'Unfamiliar sign-in properties'
EARLIEST=-30d

### Normalize and Relate Entities

In [ ]:
entities_df = alerts_dataframe.pipe(normalize_entities).pipe(relate_entities)
entities_df = entities_df[entities_df["taegis_magic.entities.field"]==GROUP_BY].reset_index(drop=True)
entities_df[[
    "tenant_id",
    "metadata.title",
    "taegis_magic.entities.field",
    "taegis_magic.entities.value",
] + [column for column in entities_df.columns if column.startswith("@") and column != GROUP_BY]]

### Generate Context Search Queries based on Entities

In [ ]:
entities_df = entities_df.pipe(generate_context_queries)
entities_df[[
    "taegis_magic.open_alerts_query",
    "taegis_magic.resolved_alerts_query",
    "taegis_magic.investigations_query",
    "taegis_magic.events_query",
]]

### Run Context Queries

In [ ]:
entity_queries = {}

for _, row in entities_df[[
    'taegis_magic.entities.value',
    'taegis_magic.open_alerts_query',
    'taegis_magic.resolved_alerts_query',
    'taegis_magic.investigations_query',
    'taegis_magic.events_query'
]].iterrows():
    # setup
    entity = row['taegis_magic.entities.value']
    open_alerts_query = row['taegis_magic.open_alerts_query']
    resolved_alerts_query = row['taegis_magic.resolved_alerts_query']
    investigations_query = row['taegis_magic.investigations_query']
    events_query = row['taegis_magic.events_query']

    # run queries
    %taegis alerts search --assign open_alerts --region $REGION --tenant $TENANT --cell "$open_alerts_query"
    %taegis alerts search --assign resolved_alerts --region $REGION --tenant $TENANT --cell "$resolved_alerts_query"
    %taegis alerts search --assign investigations --region $REGION --tenant $TENANT --cell "$investigations_query"
    %taegis events search --assign events --region $REGION --tenant $TENANT --cell "$events_query"

    # relate back to entity
    entity_queries[entity] = {}
    entity_queries[entity]['open_alerts'] = open_alerts
    entity_queries[entity]['resolved_alerts'] = resolved_alerts
    entity_queries[entity]['investigations'] = investigations
    entity_queries[entity]['events'] = events

### Display Results

In [ ]:
display_facets(entity_queries)